In [354]:
import pandas as pd 
import numpy as np
df = pd.read_csv("Bishkek.csv")
df

,price,m2_price,square,rooms,district,micro_district,building_type,floor,floors,year,date,source,condition
0,110880,720,154,4,Октябрьский район,Магистраль,кирпичный,12,14,2023.0,2023-02-04 20:05:01,Site,под самоотделку (ПСО)
1,57000,740,77,2,Октябрьский район,Магистраль,кирпичный,5,12,2023.0,2023-02-04 20:04:49,Site,под самоотделку (ПСО)
2,57750,750,77,2,Октябрьский район,Магистраль,монолитный,5,12,2023.0,2023-02-04 20:04:37,Site,под самоотделку (ПСО)
3,57000,740,77,2,Октябрьский район,Магистраль,кирпичный,5,12,2023.0,2023-02-04 20:04:01,Site,под самоотделку (ПСО)
4,55000,1279,43,1,Ленинский район,Академия Наук,кирпичный,1,12,2013.0,2023-02-04 20:02:37,Site,хорошее
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,45000,881,51,1,Октябрьский район,NaN,панельный,9,9,2021.0,2022-11-30 00:40:24,Android,евроремонт
9996,150000,1456,103,3,Октябрьский район,Магистраль,монолитный,3,12,2019.0,2022-11-30 00:03:22,Site,NaN
9997,120000,1143,105,5,Ленинский район,Московская - Белинка,кирпичный,5,6,1990.0,2022-11-29 23:52:23,Site,хорошее
9998,62000,969,64,2,Октябрьский район,Тунгуч м-н,монолитный,6,12,NaN,2022-11-29 23:21:20,Android,евроремонт


## 1. Data Collection

To build a reliable house price prediction model, it is essential to collect high-quality and relevant datasets.  
For this project, I focused on gathering data related to **real estate in Kyrgyzstan**, particularly in **Bishkek**,  
as well as supporting datasets that reflect **economic** and **demographic indicators** that may influence housing prices.

---

### 🔹 Main Dataset

- **Source**: *Kaggle / Local Open Data Portals / Real Estate Websites*  
- **Filename**: `Bishkek.csv`  
- **Description**: This dataset contains listings of apartments for sale in Bishkek.  
  It includes the following features:

| Column Name       | Description                                                 |
|-------------------|-------------------------------------------------------------|
| `price`           | Total price of the apartment (in KGS)                       |
| `m2_price`        | Price per square meter                                      |
| `square`          | Total area of the apartment in square meters                |
| `rooms`           | Number of rooms                                             |
| `district`        | Administrative district of Bishkek                          |
| `micro_district`  | Sub-district or neighborhood within the city                |
| `building_type`   | Type of the building (e.g., panel, brick, monolithic)       |
| `floor`           | The floor the apartment is located on                       |
| `floors`          | Total number of floors in the building                      |
| `year`            | Year the building was constructed                           |
| `date`            | Date the listing was posted                                 |
| `source`          | Source website or platform from which the data was collected |
| `condition`       | Condition of the apartment (e.g., renovated, under repair)  |

---

This dataset provides detailed property-level information,  
which is essential for analyzing trends in the real estate market and for building accurate predictive models.

# Data Preprocessing:

In [330]:
# Check for gaps and duplicates
print("\nNumber of missing values:")
print(df.isnull().sum())

print("\nNumber of dublicates:")
print(df.duplicated().sum())
df = df.drop_duplicates()



Number of missing values:
price                0
m2_price             0
square               0
rooms                0
district            10
micro_district     457
building_type        0
floor                0
floors               0
year              4369
date                 0
source               0
condition         2919
dtype: int64

Number of dublicates:
0


In [331]:
df = df.dropna(subset=['district'])
df.drop(columns=['year'], inplace=True)#не нужный
df.drop(columns=["source","date"], inplace=True)# не нужный

df["micro_district"].value_counts()
df["condition"] = df["condition"].fillna(df["condition"].mode()[0])# заполнил модой nan значения
df["micro_district"] = df["micro_district"].fillna(df["micro_district"].mode()[0]) # заполнил модой nan значения

In [332]:
# Check for gaps and duplicates
print("\nNumber of missing values:")
print(df.isnull().sum())

print("\nNumber of dublicates:")
print(df.duplicated().sum())
df = df.drop_duplicates()


Number of missing values:
price             0
m2_price          0
square            0
rooms             0
district          0
micro_district    0
building_type     0
floor             0
floors            0
condition         0
dtype: int64

Number of dublicates:
1299


In [333]:
df["building_type"].unique()

array(['кирпичный', 'монолитный', 'панельный'], dtype=object)

In [334]:
df.groupby('building_type')['m2_price'].mean().loc[['кирпичный', 'монолитный', 'панельный']]

building_type
кирпичный     1028.153471
монолитный    1037.822861
панельный     1074.759807
Name: m2_price, dtype: float64

In [335]:
df['building_type'].value_counts()

building_type
монолитный    3963
кирпичный     2867
панельный     1861
Name: count, dtype: int64

##  Building Type vs. Price:
### Interestingly, panel buildings turned out to be the most expensive in terms of average price per square meter — which goes against what people usually expect.

### Here’s the breakdown:
	•	Panel: 1074.76 KGS/m²
	•	Monolithic: 1037.82 KGS/m²
	•	Brick: 1028.15 KGS/m²

### The difference isn’t huge, but it’s still surprising. This could be due to the location of these buildings or the condition of the apartments.

### Also, the number of listings for each type is large enough (1800+ each), so the results are reliable.
### So we can convert them into a numeric format for our model (panel=3, monolithic=2, brick=1)

In [336]:
df['building_type'] = df['building_type'].apply(
    lambda x: 1 if x == 'кирпичный' else (2 if x == 'монолитный' else (3 if x == 'панельный' else x))
)

In [337]:
df["condition"].unique()

array(['под самоотделку (ПСО)', 'хорошее', 'евроремонт', 'среднее',
       'не достроено', 'требует ремонта', 'черновая отделка',
       'свободная планировка'], dtype=object)

In [338]:
df.groupby('condition')['m2_price'].mean().sort_values(ascending=False)

condition
свободная планировка     1092.900000
евроремонт               1089.771561
хорошее                  1077.003454
среднее                  1063.333966
требует ремонта          1009.889535
под самоотделку (ПСО)     885.744942
черновая отделка          843.485714
не достроено              751.842105
Name: m2_price, dtype: float64

In [339]:
df['condition'] = df['condition'].apply(lambda x: 0 if x == 'не достроено' else
                                         1 if x == 'требует ремонта' else
                                         2 if x == 'черновая отделка' else
                                         3 if x == 'под самоотделку (ПСО)' else
                                         6 if x == 'свободная планировка' else
                                         4 if x == 'среднее' else
                                         5 if x == 'хорошее' else
                                         7 if x == 'евроремонт' else None)



In [340]:
df["micro_district"].unique()

array(['Магистраль', 'Академия Наук', 'ЖД вокзал', 'Аламедин-1 м-н',
       '6 м-н', 'Кок-Жар ж/м', 'Асанбай м-н', 'Джал-23 м-н (Нижний Джал)',
       'Военторг', 'АЮ Grand', 'Восток-5 м-н', 'Молодая Гвардия',
       'Верхний Джал м-н', 'КНУ', '4 м-н', 'Политех', 'Джал 15 м-н',
       'Ипподром', 'Площадь Победы', '11 м-н', 'Мед. академия', 'Ак Кеме',
       'Моссовет', 'Горького - Панфилова', 'Московская - Белинка',
       'Старый аэропорт', 'АУЦА', 'Дворец спорта', '12 м-н', 'Гоин',
       'Московская - Уметалиева', 'Парк Ататюрк', 'Жилгородок Ницца',
       'Карла Маркса', 'ЦУМ', 'Сквер Тоголок Молдо', 'Бишкек-Парк',
       'Душанбинка', 'Восточный автовокзал', 'Центральная мечеть',
       'Юбилейка', 'Космос', '8 м-н', 'Кара-Жыгач ж/м',
       'Джальская больница', 'Средний Джал м-н', 'Золотой квадрат',
       'Ден Сяопина - Фучика', 'Нижний Токольдош', '5 м-н', 'Матросова',
       'Парк Панфилова/Спартак', '7 м-н', 'Карпинка', 'Кудайберген',
       'Джал-29 м-н', 'Улан м-н', 'Пишп

In [341]:
# Преобразуем микрорайоны в числа
def assign_district_group(district):
    if district in ['Золотой квадрат', 'Асанбай м-н', 'Бишкек-Парк', 'ЦУМ', 'Орозбекова - Жибек-Жолу', 
                    'Горького - Панфилова', 'Московская - Белинка', 'Московская - Уметалиева', 
                    'Парк Панфилова/Спартак', 'Филармония']:
        return 4  # Самые дорогие
    elif district in ['Академия Наук', 'КНУ', 'АУЦА', 'Дворец спорта', 'Мед. академия', 'Ак Кеме', 
                      'Молодая Гвардия', 'Восток-5 м-н', 'Юг-2 м-н', 'Кок-Жар м-н', 'Центральная мечеть', 
                      'Парк Ататюрк', 'Ген прокуратура']:
        return 3  # Высокая цена
    elif district in ['4 м-н', '5 м-н', '6 м-н', '7 м-н', '8 м-н', '9 м-н', '10 м-н', '11 м-н', '12 м-н', 
                      'Джал 15 м-н', 'Джал-23 м-н (Нижний Джал)', 'Джал-29 м-н', 'Джал-30 м-н', 'Средний Джал м-н', 
                      'Верхний Джал м-н', 'Улан м-н', 'Тунгуч м-н', 'Аламедин-1 м-н', 'Политех', 'КГУСТА', 
                      'БГУ', 'Азия Молл', 'Советская - Скрябина', 'Карпинка', 'Матросова']:
        return 2  # Средняя цена
    elif district in ['ЖД вокзал', 'Военторг', 'АЮ Grand', 'Ипподром', 'Площадь Победы', 'Моссовет', 
                      'Старый аэропорт', 'Гоин', 'Душанбинка', 'Восточный автовокзал', 'Юбилейка', 'Космос', 
                      'Кара-Жыгач ж/м', 'Джальская больница', 'Нижний Токольдош', 'Кудайберген', 'Пишпек ж/м', 
                      'ТЭЦ', 'VEFA', 'Щербакова ж/м', 'Ак Эмир рынок', 'Госрегистр', 'Церковь', 'Чуй - Алматинка', 
                      'Западный автовокзал', 'Таатан', 'Гагарина', 'Учкун м-н', 'Городок энергетиков', 
                      'Аламединский рынок', 'Жилгородок Совмина ж/м', 'Улан-2 м-н']:
        return 1  # Ниже среднего
    else:
        return 0  # Самые дешевые (жилмассивы)

# Применяем функцию ко всем микрорайонам в DataFrame
df['micro_district'] = df['micro_district'].apply(assign_district_group)

# Проверим результат
print(df.head(10))

     price  m2_price  square rooms            district  micro_district  \
0   110880       720     154     4   Октябрьский район               0   
1    57000       740      77     2   Октябрьский район               0   
2    57750       750      77     2   Октябрьский район               0   
4    55000      1279      43     1     Ленинский район               3   
5    97000      1276      76     2  Первомайский район               1   
6   160000      1455     110     3  Первомайский район               0   
7   158990       880     181     4   Октябрьский район               0   
8    68000      1283      53     1   Октябрьский район               0   
9    57500      1198      48     2  Свердловский район               2   
10   60000      1364      44     1   Октябрьский район               0   

    building_type  floor  floors  condition  
0               1     12      14          3  
1               1      5      12          3  
2               2      5      12          3  
4

In [342]:
df

,price,m2_price,square,rooms,district,micro_district,building_type,floor,floors,condition
0,110880,720,154,4,Октябрьский район,0,1,12,14,3
1,57000,740,77,2,Октябрьский район,0,1,5,12,3
2,57750,750,77,2,Октябрьский район,0,2,5,12,3
4,55000,1279,43,1,Ленинский район,3,1,1,12,5
5,97000,1276,76,2,Первомайский район,1,1,10,12,7
...,...,...,...,...,...,...,...,...,...,...
9994,105000,1346,78,4,Октябрьский район,2,2,8,10,7
9995,45000,881,51,1,Октябрьский район,0,3,9,9,7
9996,150000,1456,103,3,Октябрьский район,0,2,3,12,7
9997,120000,1143,105,5,Ленинский район,4,1,5,6,5


In [343]:
df["district"].unique()

array(['Октябрьский район', 'Ленинский район', 'Первомайский район',
       'Свердловский район'], dtype=object)

In [344]:
# Группируем данные по району и рассчитываем среднюю цену для каждого района
average_price_per_district = df.groupby('district')['price'].mean()

# Сортируем по средней цене (по убыванию)
sorted_average_price = average_price_per_district.sort_values(ascending=False)

# Выводим результат
print(sorted_average_price)

district
Первомайский район    84276.056991
Ленинский район       75635.168817
Октябрьский район     75186.521700
Свердловский район    63677.534359
Name: price, dtype: float64


In [345]:
def assign_district_price_rating(district):
    if district == 'Первомайский район':
        return 3
    elif district == 'Ленинский район':
        return 2
    elif district == 'Октябрьский район':
        return 1
    elif district == 'Свердловский район':
        return 0
    else:
        return None
df['district'] = df['district'].apply(assign_district_price_rating)

df

,price,m2_price,square,rooms,district,micro_district,building_type,floor,floors,condition
0,110880,720,154,4,1,0,1,12,14,3
1,57000,740,77,2,1,0,1,5,12,3
2,57750,750,77,2,1,0,2,5,12,3
4,55000,1279,43,1,2,3,1,1,12,5
5,97000,1276,76,2,3,1,1,10,12,7
...,...,...,...,...,...,...,...,...,...,...
9994,105000,1346,78,4,1,2,2,8,10,7
9995,45000,881,51,1,1,0,3,9,9,7
9996,150000,1456,103,3,1,0,2,3,12,7
9997,120000,1143,105,5,2,4,1,5,6,5


In [346]:
df['floor/floors'] = df['floor'] / df['floors']
df.drop(columns=['floor','floors'], inplace=True)#не нужный
df

,price,m2_price,square,rooms,district,micro_district,building_type,condition,floor/floors
0,110880,720,154,4,1,0,1,3,0.857143
1,57000,740,77,2,1,0,1,3,0.416667
2,57750,750,77,2,1,0,2,3,0.416667
4,55000,1279,43,1,2,3,1,5,0.083333
5,97000,1276,76,2,3,1,1,7,0.833333
...,...,...,...,...,...,...,...,...,...
9994,105000,1346,78,4,1,2,2,7,0.800000
9995,45000,881,51,1,1,0,3,7,1.000000
9996,150000,1456,103,3,1,0,2,7,0.250000
9997,120000,1143,105,5,2,4,1,5,0.833333


In [347]:
df.to_csv('Bishkek_Preprocessed.csv', index=False)

In [348]:
data = pd.read_csv('Bishkek_Preprocessed.csv')


,price,m2_price,square,rooms,district,micro_district,building_type,condition,floor/floors
0,110880,720,154,4,1,0,1,3,0.857143
1,57000,740,77,2,1,0,1,3,0.416667
2,57750,750,77,2,1,0,2,3,0.416667
3,55000,1279,43,1,2,3,1,5,0.083333
4,97000,1276,76,2,3,1,1,7,0.833333
...,...,...,...,...,...,...,...,...,...
8686,105000,1346,78,4,1,2,2,7,0.800000
8687,45000,881,51,1,1,0,3,7,1.000000
8688,150000,1456,103,3,1,0,2,7,0.250000
8689,120000,1143,105,5,2,4,1,5,0.833333
